In [1]:
import os
import pandas as pd
import numpy as np

Setting up the enviornment

In [2]:
### this cell separates the subjects and session of the frame displacement data
abspath = os.path.abspath('__file__')
dname = os.path.dirname(os.path.dirname(abspath))
os.environ["ROOTDIR"] = dname  # seth path
rootdir = os.environ["ROOTDIR"]

metadata_path = os.path.join(rootdir, 'metadata')

Reading the subjects in the first wave

In [3]:
data = pd.read_csv(os.path.join(metadata_path,'sub_list_fd.csv'))
## solitting the file name to sub ses func and 
sub_ses_df = data['file_name'].str.split('/', expand=True).drop([0,1,2,3,4,5,8,9],axis=1)
## remove the file name of the actual data remaining the fd checking axis =1 means collunm
fd_df = data.drop(['file_name'],axis=1)
#concatenating the final data
fd_data = pd.concat([sub_ses_df,fd_df],axis=1)
fd_data = fd_data.rename(columns={6:'sub_id' , 7:'session'})
fd_data.drop(fd_data[fd_data.session == 'ses-3'].index, inplace=True)
fd_data.drop(fd_data[fd_data.session == 'ses-2'].index, inplace=True)
fd_data = fd_data.sort_values("sub_id") 

###### IMPORTANT: here false is optimal in both conditions

In [ ]:
#READING THE DATA
confund_data = pd.read_csv(os.path.join(metadata_path, 'checaCONFOUNDS.txt'), sep= " ")
sub_ids = pd.read_csv(os.path.join(metadata_path, 'subjects.txt'),sep=' ')
#remove the unnecessary file names and indexes
sub_ids = sub_ids['FILE_NAME'].str.split('/', expand=True).drop([2,3],axis=1)
confund_data = confund_data.drop(['index','V2'], axis=1)

conf_sub_data = pd.concat([sub_ids, confund_data['V1']==181], axis=1)
conf_sub_data = conf_sub_data.rename({0 : 'sub_id', 1:'session'}, axis=1)
#dopping other sessions / wave
conf_sub_data.drop(conf_sub_data[conf_sub_data.session == 'ses-3'].index, inplace=True)
conf_sub_data.drop(conf_sub_data[conf_sub_data.session == 'ses-2'].index, inplace=True)
conf_sub_data

,sub_id,session,V1
0,sub-00003,ses-1,True
2,sub-00008,ses-1,True
3,sub-00015,ses-1,True
4,sub-00016,ses-1,True
7,sub-00019,ses-1,True
...,...,...,...
1514,sub-02496,ses-1,True
1519,sub-02502,ses-1,True
1522,sub-02507,ses-1,True
1523,sub-02509,ses-1,True


In [ ]:
df2 = pd.merge(fd_data, conf_sub_data, on=['sub_id','session'])
df2 = df2.drop(['session'], axis=1)
df2

,sub_id,fd_mean,fd_mean_value,fd_count_high,fd_count_high_value,V1
0,sub-00003,False,0.072094,False,3,True
1,sub-00008,True,0.311252,True,51,True
2,sub-00015,False,0.118437,False,3,True
3,sub-00016,True,0.587256,True,101,True
4,sub-00019,False,0.129256,False,17,True
...,...,...,...,...,...,...
663,sub-02496,False,0.131778,True,21,True
664,sub-02502,False,0.157067,False,16,True
665,sub-02507,False,0.096652,False,8,True
666,sub-02509,False,0.092963,False,2,True


Uploading sociodemografic and psicometric data and doing random sample imputation for the missing data

In [6]:
#Reading the data
socidemo_path = os.path.join(rootdir,'fMRI_INPD_sociodemographic')
socio_psi = pd.read_excel(os.path.join(socidemo_path,'banco_angelo.xlsx'))

dados_clinicos = socio_psi[['ident', 'gender', 'abepscore','trauma_exp', 'age', 'DQ','subjectid', 'cape_tot', 'cape_PA_score', 'cape_PI_score', 'cape_BE_score']]
# dropping subjects that dont ha cape_tot value
dados_clinicos = dados_clinicos[dados_clinicos['cape_tot'].notna()]
# ramdom sample
def random_sample_imputation(df):
   
    cols_with_missing_values = df.columns[df.isna().any()].tolist()

    for var in cols_with_missing_values:

        # extract a random sample
        random_sample_df = df[var].dropna().sample(df[var].isnull().sum(),
                                                    random_state=0, replace=True)
        # re-index the randomly extracted sample
        random_sample_df.index = df[
                df[var].isnull()].index

        # replace the NA
        df.loc[df[var].isnull(), var] = random_sample_df
    
    return df

dados_clinicos = random_sample_imputation(dados_clinicos)
dados_clinicos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2249 entries, 2 to 2509
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ident          2249 non-null   int64  
 1   gender         2249 non-null   int64  
 2   abepscore      2249 non-null   int64  
 3   trauma_exp     2249 non-null   float64
 4   age            2249 non-null   float64
 5   DQ             2249 non-null   float64
 6   subjectid      2249 non-null   int64  
 7   cape_tot       2249 non-null   float64
 8   cape_PA_score  2249 non-null   float64
 9   cape_PI_score  2249 non-null   float64
 10  cape_BE_score  2249 non-null   float64
dtypes: float64(7), int64(4)
memory usage: 210.8 KB


In [7]:
for index in dados_clinicos['ident']:  
    dados_clinicos['ident'] = dados_clinicos['ident'].replace(index, 'sub-{}'.format(str(index).zfill(5)))
dados_clinicos = dados_clinicos.rename(columns={'ident':'sub_id'})

merging neuroimage masures to psicometric and sociodemo dataframes

In [8]:
df_final = pd.merge(dados_clinicos, df2, on=['sub_id'])
df_final = df_final.query('V1 == True')
df_final

,sub_id,gender,abepscore,trauma_exp,age,DQ,subjectid,cape_tot,cape_PA_score,cape_PI_score,cape_BE_score,fd_mean,fd_mean_value,fd_count_high,fd_count_high_value,V1
0,sub-00003,2,10,0.0,8.361396,94.0,10059,1.0,0.0,1.0,0.0,False,0.072094,False,3,True
1,sub-00008,1,16,0.0,12.569473,76.0,10103,7.0,0.0,2.0,2.0,True,0.311252,True,51,True
2,sub-00015,2,19,0.0,12.221766,85.0,10134,5.0,0.0,4.0,0.0,False,0.118437,False,3,True
3,sub-00016,1,10,1.0,11.619439,100.0,10210,1.0,0.0,1.0,0.0,True,0.587256,True,101,True
4,sub-00019,2,13,0.0,11.049966,75.0,10713,4.0,0.0,2.0,0.0,False,0.129256,False,17,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,sub-02496,1,17,0.0,8.038330,86.0,20294,12.0,2.0,5.0,3.0,False,0.131778,True,21,True
662,sub-02502,1,23,1.0,9.911020,88.0,21004,6.0,0.0,2.0,0.0,False,0.157067,False,16,True
663,sub-02507,1,19,0.0,8.440794,106.0,20046,4.0,0.0,1.0,1.0,False,0.096652,False,8,True
664,sub-02509,1,14,0.0,8.933607,109.0,20671,14.0,0.0,3.0,4.0,False,0.092963,False,2,True


Removing subjects due to excluion criteria

In [16]:
# dealing with manual inspection 
# Open the file and read the content
with open(os.path.join(metadata_path,'subid_imagem-ruim.txt'), 'r') as file:
    content = file.read()

# Split based on commas (or any other delimiter)
variables = content.split(' ')  # Adjust the delimiter as needed

# Remover entradas vazias, caso existam
variables = [s for s in variables if s]

# Criar uma lista de listas fazendo o split
split_data = [s.split('_') for s in variables]

# Criar o DataFrame com base nos resultados do split
manIsnpec = pd.DataFrame(split_data, columns=['sub_id', 'session', 'func', 'sub_id_2', 'session_2', 'task', 'run', 'space', 'resolution', 'description', 'new'])

# Organizing the dataset for only selecting impartant features, such as w0 images aka ses-1
manIsnpec = manIsnpec.drop(['func', 'sub_id_2', 'session_2', 'task', 'run', 'space', 'resolution', 'description', 'new'], axis=1)
manIsnpec = manIsnpec.query('session == "ses-1"')
df_final = pd.merge(df_final, manIsnpec, on='sub_id', how='outer')


In [10]:
# dropping the subjects that have to be removed by qualy control of images
threshold_fdmean, threshold_fdcount, threshold_DQ = 0.3, 30, 70

df_final_fdmean = df_final.query(f'fd_mean<={threshold_fdmean}')
print(df_final_fdmean.shape[0])
df_final_fdhigh = df_final.query(f'fd_count_high <={threshold_fdcount}')
print(df_final_fdhigh.shape[0])
df_final_dq = df_final.query(f'DQ >= {threshold_DQ}')
print(df_final_dq.shape[0])

df_final_final = df_final.query(f'fd_mean<={threshold_fdmean} and fd_count_high <={threshold_fdcount} and DQ >= {threshold_DQ}')
df_final_final = df_final_final.drop(['fd_mean', 'fd_mean_value', 'fd_count_high', 'V1', 'DQ', 'trauma_exp'], axis=1)

# identifying the site of colection
df_final_final['subjectid'] = np.floor(df_final['subjectid']/10000)
df_final_final.rename(columns={'subjectid':'colection_site'}, inplace=True)


470
663
653


comparing the databases after exclusion

In [11]:
import matplotlib.pyplot as plt

# Definir a figura
plt.figure(figsize=(12,12))

# Lista de títulos para cada DataFrame
titles = ['df_final', 'df_final_fdhigh', 'df_final_fdmean', 'df_final_maninspect']

# Iterar sobre os DataFrames e adicionar os gráficos
for idf, data_frame in enumerate([df_final, df_final_fdhigh, df_final_fdmean, df_final_maninspect]):
    for indexx, var in enumerate(['gender', 'abepscore', 'trauma_exp', 'age', 'DQ',
                                  'subjectid', 'cape_tot', 'cape_PA_score', 'cape_PI_score', 'cape_BE_score']):
        
        # Subplot: 4 linhas, 10 colunas
        plt.subplot(4, 10, 10*idf + indexx+1)
        
        # Adicionar o histograma para a variável
        plt.title(var, fontsize=8)  # Título de cada gráfico
        plt.hist(data_frame[var])
        
        # Remover o eixo vertical e os valores do gráfico
        plt.gca().axes.get_xaxis().set_visible(False)  # Esconde o eixo Y
        plt.gca().spines['left'].set_visible(False)    # Remove a linha do eixo Y
        plt.gca().spines['right'].set_visible(False)   # Remove a linha do eixo Y
        plt.gca().spines['top'].set_visible(False)     # Remove a linha do eixo Y
        plt.gca().spines['bottom'].set_visible(False)  # Remove a linha do eixo Y
    
    # Adicionar título para cada linha (DataFrame) com suptitle
    plt.suptitle(titles[idf], fontsize=14, weight='bold', y=1-(idf * 0.25))  # Ajustar a posição com 'y'

# Ajustar o layout para evitar sobreposição
plt.tight_layout(rect=[0, 0, 1, 0.97])  # Deixar espaço no topo para os títulos
plt.show()


NameError: name 'df_final_maninspect' is not defined

<Figure size 1200x1200 with 0 Axes>

Exporting

In [10]:
# df_final.to_excel(os.path.join(metadata_path,'variaveis_analise_conf.xlsx'))
# df_final.shape[0]

In [11]:
# n = df_final[['gender', 'abepscore']]